In [4]:
import pandas as pd
pd.set_option('display.max_columns', None)


In [5]:
table_PS = pd.read_csv("data/PS_2024.10.05_09.10.06.csv", skiprows=96)
confiavel_PS = table_PS[table_PS["default_flag"]==1]

In [6]:
confiavel_PS = confiavel_PS.drop("pl_radj", axis=1)
confiavel_PS = confiavel_PS.drop("pl_radjerr1", axis=1)
confiavel_PS = confiavel_PS.drop("pl_radjerr2", axis=1)
confiavel_PS = confiavel_PS.drop("pl_radjlim", axis=1)
confiavel_PS = confiavel_PS.drop("pl_bmassj", axis=1)
confiavel_PS = confiavel_PS.drop("pl_bmassjerr1", axis=1)
confiavel_PS = confiavel_PS.drop("pl_bmassjerr2", axis=1)
confiavel_PS = confiavel_PS.drop("pl_bmassjlim", axis=1)

In [7]:
#colunas com niveis de insolacao onde pode ser que o planeta esteja numa possivel zona habitavel
habitaveis = confiavel_PS.query("0.7 <= pl_insol <= 1.5")
habitaveis

,loc_rowid,pl_name,hostname,default_flag,sy_snum,sy_pnum,discoverymethod,disc_year,disc_facility,soltype,pl_controv_flag,pl_refname,pl_orbper,pl_orbpererr1,pl_orbpererr2,pl_orbperlim,pl_orbsmax,pl_orbsmaxerr1,pl_orbsmaxerr2,pl_orbsmaxlim,pl_rade,pl_radeerr1,pl_radeerr2,pl_radelim,pl_bmasse,pl_bmasseerr1,pl_bmasseerr2,pl_bmasselim,pl_bmassprov,pl_orbeccen,pl_orbeccenerr1,pl_orbeccenerr2,pl_orbeccenlim,pl_insol,pl_insolerr1,pl_insolerr2,pl_insollim,pl_eqt,pl_eqterr1,pl_eqterr2,pl_eqtlim,ttv_flag,st_refname,st_spectype,st_teff,st_tefferr1,st_tefferr2,st_tefflim,st_rad,st_raderr1,st_raderr2,st_radlim,st_mass,st_masserr1,st_masserr2,st_masslim,st_met,st_meterr1,st_meterr2,st_metlim,st_metratio,st_logg,st_loggerr1,st_loggerr2,st_logglim,sy_refname,rastr,ra,decstr,dec,sy_dist,sy_disterr1,sy_disterr2,sy_vmag,sy_vmagerr1,sy_vmagerr2,sy_kmag,sy_kmagerr1,sy_kmagerr2,sy_gaiamag,sy_gaiamagerr1,sy_gaiamagerr2,rowupdate,pl_pubdate,releasedate
511,512,GJ 1061 c,GJ 1061,1,1,3,Radial Velocity,2020,La Silla Observatory,Published Confirmed,0,<a refstr=DREIZLER_ET_AL__2020 href=https://ui...,6.689000,0.005000,-0.005000,0.0,0.035000,0.001000,-0.001000,0.0,NaN,NaN,NaN,NaN,1.74000,0.23000,-0.23000,0.0,Msini,0.290,NaN,NaN,1.0,1.40,0.20,-0.20,0.0,NaN,NaN,NaN,NaN,0,<a refstr=DREIZLER_ET_AL__2020 href=https://ui...,M5.5 V,2953.00,98.00,-98.00,0.0,0.16,0.01,-0.01,0.0,0.12,0.01,-0.01,0.0,-0.080,0.080,-0.080,0.0,[Fe/H],NaN,NaN,NaN,NaN,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,03h36m00.78s,54.003249,-44d30m51.52s,-44.514310,3.672780,0.000890,-0.000900,12.700,0.200,-0.200,6.610,0.021,-0.021,10.98890,0.000945,-0.000945,2020-02-21,2020-03,2020-02-27
583,584,GJ 273 b,GJ 273,1,1,2,Radial Velocity,2017,La Silla Observatory,Published Confirmed,0,<a refstr=ASTUDILLO_DEFRU_ET_AL__2017 href=htt...,18.649800,0.005900,-0.005200,0.0,0.091101,0.000019,-0.000017,0.0,NaN,NaN,NaN,NaN,2.89000,0.27000,-0.26000,0.0,Msini,0.100,0.090000,-0.070000,0.0,1.06,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0,<a refstr=ASTUDILLO_DEFRU_ET_AL__2017 href=htt...,M3.5,3382.00,49.00,-49.00,0.0,0.29,0.03,-0.03,0.0,0.29,NaN,NaN,0.0,0.090,0.170,-0.170,0.0,[Fe/H],NaN,NaN,NaN,NaN,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,07h27m24.50s,111.852091,+05d13m32.91s,5.225807,5.921535,0.020633,-0.020633,9.840,0.100,-0.100,4.857,0.023,-0.023,NaN,NaN,NaN,2017-03-29,2017-06,2017-03-30
605,606,GJ 3293 b,GJ 3293,1,1,4,Radial Velocity,2015,La Silla Observatory,Published Confirmed,0,<a refstr=ASTUDILLO_DEFRU_ET_AL__2017 href=htt...,30.598700,0.008300,-0.008400,0.0,0.143390,0.000030,-0.000030,0.0,NaN,NaN,NaN,NaN,23.54000,0.88000,-0.89000,0.0,Msini,0.060,0.040000,-0.040000,0.0,1.07,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0,<a refstr=ASTUDILLO_DEFRU_ET_AL__2017 href=htt...,M2.5,3466.00,49.00,-49.00,0.0,0.40,0.03,-0.03,0.0,0.42,NaN,NaN,0.0,0.020,0.090,-0.090,0.0,[Fe/H],NaN,NaN,NaN,NaN,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,04h28m35.63s,67.148443,-25d10m16.82s,-25.171340,20.190300,0.021600,-0.021600,11.945,0.037,-0.037,7.486,0.033,-0.033,10.97880,0.000878,-0.000878,2017-03-29,2017-06,2017-03-30
1902,1903,HD 109286 b,HD 109286,1,1,1,Radial Velocity,2021,Haute-Provence Observatory,Published Confirmed,0,<a refstr=DEMANGEON_ET_AL__2021 href=https://u...,520.100000,2.300000,-2.300000,0.0,1.259000,0.021000,-0.022000,0.0,NaN,NaN,NaN,NaN,950.30694,47.67426,-47.67426,0.0,Msini,0.338,0.034000,-0.035000,0.0,0.71,0.06,-0.06,0.0,259.0,6.0,-6.0,0.0,0,<a refstr=DEMANGEON_ET_AL__2021 href=https://u...,G4 V,5694.00,23.00,-23.00,0.0,1.09,0.04,-0.04,0.0,0.98,0.05,-0.05,0.0,0.050,0.020,-0.020,0.0,[Fe/H],4.44,0.04,-0.04,0.0,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,12h33m35.00s,188.395842,+07d16m51.26s,7.280906,55.415600,0.157000,-0.156200,8.780,0.030,-0.030,7.162,0.029,-0.029,8.61599,0.000595,-0.000595,2021-07-26,2021-09,2021-07-26
2646,2647,HD 191939 g,HD 191939,1,1,6,Radial Velocity,2022,Multiple Observatories,Published Confirmed,0,<a refstr=ORELL_MIQUEL_ET_AL__2022 href=https:...,284.000000,10.000000,-8.000000,0.0,0.812000,0.028

In [8]:
#ordenando os valores do dataset com base em sua orbita
# tende a 0 - orbita mais circular
# entre - orbita mais eliptica
# tende a 1 - orbita mais hiperbolica
# ignora nulos (na coluna)
orbita = confiavel_PS[confiavel_PS["pl_orbeccen"].notna()].sort_values(by="pl_orbeccen")
orbita

,loc_rowid,pl_name,hostname,default_flag,sy_snum,sy_pnum,discoverymethod,disc_year,disc_facility,soltype,pl_controv_flag,pl_refname,pl_orbper,pl_orbpererr1,pl_orbpererr2,pl_orbperlim,pl_orbsmax,pl_orbsmaxerr1,pl_orbsmaxerr2,pl_orbsmaxlim,pl_rade,pl_radeerr1,pl_radeerr2,pl_radelim,pl_bmasse,pl_bmasseerr1,pl_bmasseerr2,pl_bmasselim,pl_bmassprov,pl_orbeccen,pl_orbeccenerr1,pl_orbeccenerr2,pl_orbeccenlim,pl_insol,pl_insolerr1,pl_insolerr2,pl_insollim,pl_eqt,pl_eqterr1,pl_eqterr2,pl_eqtlim,ttv_flag,st_refname,st_spectype,st_teff,st_tefferr1,st_tefferr2,st_tefflim,st_rad,st_raderr1,st_raderr2,st_radlim,st_mass,st_masserr1,st_masserr2,st_masslim,st_met,st_meterr1,st_meterr2,st_metlim,st_metratio,st_logg,st_loggerr1,st_loggerr2,st_logglim,sy_refname,rastr,ra,decstr,dec,sy_dist,sy_disterr1,sy_disterr2,sy_vmag,sy_vmagerr1,sy_vmagerr2,sy_kmag,sy_kmagerr1,sy_kmagerr2,sy_gaiamag,sy_gaiamagerr1,sy_gaiamagerr2,rowupdate,pl_pubdate,releasedate
33851,33852,TOI-1347 c,TOI-1347,1,1,2,Transit,2024,Transiting Exoplanet Survey Satellite (TESS),Published Confirmed,0,<a refstr=RUBENZAHL_ET_AL__2024 href=https://u...,4.841962,1.200000e-05,-1.200000e-05,0.0,NaN,NaN,NaN,NaN,1.600,0.100,-0.100,0.0,6.40000,NaN,NaN,1.0,Mass,0.0000,NaN,NaN,0.0,NaN,NaN,NaN,NaN,1000.0,25.0,-25.0,0.0,0,<a refstr=RUBENZAHL_ET_AL__2024 href=https://u...,NaN,5464.00,100.00,-100.00,0.0,0.83,0.03,-0.03,0.0,0.91,0.03,-0.03,0.0,0.040,0.060,-0.060,0.0,[Fe/H],4.64,0.10,-0.10,0.0,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,18h41m18.40s,280.326681,+70d17m24.19s,70.290054,147.4760,1.0500,-1.0350,11.168,0.013,-0.013,9.616,0.015,-0.015,11.20870,0.001150,-0.001150,2024-03-04,2024-02,2024-03-04
6570,6571,Kepler-101 c,Kepler-101,1,1,2,Transit,2014,Kepler,Published Confirmed,0,<a refstr=BONOMO_ET_AL__2014 href=https://ui.a...,6.029760,7.500000e-05,-7.500000e-05,0.0,0.06840,0.00140,-0.00140,0.0,1.250,0.190,-0.170,0.0,3.78000,NaN,NaN,1.0,Mass,0.0000,NaN,NaN,0.0,NaN,NaN,NaN,NaN,1413.0,238.0,-210.0,0.0,0,<a refstr=BONOMO_ET_AL__2014 href=https://ui.a...,G3 IV,5667.00,50.00,-50.00,0.0,1.56,0.20,-0.20,0.0,1.17,0.07,-0.05,0.0,0.330,0.070,-0.070,0.0,[Fe/H],4.12,0.11,-0.09,0.0,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,18h53m01.32s,283.255507,+48d21m18.89s,48.355247,927.9810,14.7985,-14.7985,13.700,0.069,-0.069,12.011,0.020,-0.020,13.56720,0.000214,-0.000214,2014-09-25,2014-12,2014-09-25
35507,35508,WASP-24 b,WASP-24,1,3,1,Transit,2010,SuperWASP,Published Confirmed,0,<a refstr=STASSUN_ET_AL__2017 href=https://ui....,2.341210,2.000000e-06,-2.000000e-06,0.0,NaN,NaN,NaN,NaN,15.468,1.793,-1.793,0.0,394.10920,92.17070,-92.17070,0.0,Mass,0.0000,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,<a refstr=STASSUN_ET_AL__2017 href=https://ui....,NaN,6075.00,100.00,-100.00,0.0,1.42,0.16,-0.16,0.0,1.43,0.50,-0.50,0.0,0.000,NaN,NaN,0.0,[Fe/H],4.26,0.17,-0.17,0.0,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,15h08m51.72s,227.215497,+02d20m35.83s,2.343286,322.1140,4.4980,-4.3780,11.219,0.017,-0.017,10.148,0.023,-0.023,11.39450,0.000432,-0.000432,2018-04-25,2017-03,2018-04-26
1986,1987,HD 116029 b,HD 116029,1,2,2,Radial Velocity,2011,W. M. Keck Observatory,Published Confirmed,0,<a refstr=STASSUN_ET_AL__2017 href=https://ui....,670.000000,1.100000e+01,-1.100000e+01,0.0,1.65000,0.05000,-0.05000,0.0,NaN,NaN,NaN,NaN,444.96200,92.17070,-92.17070,0.0,Msini,0.0000,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,<a refstr=STASSUN_ET_AL__2017 href=https://ui....,NaN,4819.00,54.00,-54.00,0.0,4.89,0.21,-0.21,0.0,0.83,0.24,-0.24,0.0,0.090,NaN,NaN,0.0,[Fe/H],2.98,0.12,-0.12,0.0,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,13h20m39.53s,200.164691,+24d38m54.50s,24.648473,123.2450,0.5880,-0.5880,7.880,0.030,-0.030,5.534,0.016,-0.016,7.60909,0.000196,-0.000196,2018-09-04,2017-03,2018-09-06
6380,6381,Kepler-10 b,Kepler-10,1,1,3,Transit,2011,Kepler,Published Confirmed,0,<a refstr=BONOMO_ET_AL__2023 href=https://ui.a...,0.837491,2.000000e-07,-2.000000e-07,0.0,0.01685,0.00013,-0.00013,0.0,1.470,0.030,-0.020,0.0,3.26000,0.3

In [9]:
PARSEC = (30857 * 10e12)

In [10]:
#criando uma coluna que transforma a distancia de parsec para km em not. cientifica
confiavel_PS["sy_dist_km"] = confiavel_PS["sy_dist"] * PARSEC

In [11]:
confiavel_PS[confiavel_PS["pl_eqt"].notna()].query("273 <= pl_eqt <= 373")

,loc_rowid,pl_name,hostname,default_flag,sy_snum,sy_pnum,discoverymethod,disc_year,disc_facility,soltype,pl_controv_flag,pl_refname,pl_orbper,pl_orbpererr1,pl_orbpererr2,pl_orbperlim,pl_orbsmax,pl_orbsmaxerr1,pl_orbsmaxerr2,pl_orbsmaxlim,pl_rade,pl_radeerr1,pl_radeerr2,pl_radelim,pl_bmasse,pl_bmasseerr1,pl_bmasseerr2,pl_bmasselim,pl_bmassprov,pl_orbeccen,pl_orbeccenerr1,pl_orbeccenerr2,pl_orbeccenlim,pl_insol,pl_insolerr1,pl_insolerr2,pl_insollim,pl_eqt,pl_eqterr1,pl_eqterr2,pl_eqtlim,ttv_flag,st_refname,st_spectype,st_teff,st_tefferr1,st_tefferr2,st_tefflim,st_rad,st_raderr1,st_raderr2,st_radlim,st_mass,st_masserr1,st_masserr2,st_masslim,st_met,st_meterr1,st_meterr2,st_metlim,st_metratio,st_logg,st_loggerr1,st_loggerr2,st_logglim,sy_refname,rastr,ra,decstr,dec,sy_dist,sy_disterr1,sy_disterr2,sy_vmag,sy_vmagerr1,sy_vmagerr2,sy_kmag,sy_kmagerr1,sy_kmagerr2,sy_gaiamag,sy_gaiamagerr1,sy_gaiamagerr2,rowupdate,pl_pubdate,releasedate,sy_dist_km
226,227,CFBDSIR J145829+101343 b,CFBDSIR J145829+101343,1,1,1,Imaging,2011,W. M. Keck Observatory,Published Confirmed,0,<a refstr=LIU_ET_AL__2011 href=https://ui.adsa...,10037.50000,2737.50000,-2737.50000,0.0,2.60000,0.30000,-0.30000,0.0,NaN,NaN,NaN,NaN,3337.10000,1430.20000,-1430.20000,0.0,Mass,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,370.0,40.0,-40.0,0.0,0,<a refstr=LIU_ET_AL__2011 href=https://ui.adsa...,NaN,580.5,24.5,-24.5,0.0,NaN,NaN,NaN,NaN,0.02,0.01,-0.01,0.0,NaN,NaN,NaN,NaN,NaN,4.72,0.28,-0.28,0.0,<a refstr=DELORME_ET_AL__2008 href=https://ui....,14h58m29.00s,224.620833,+10d13m43.0s,10.228611,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-05-14,2011-10,2014-05-14,NaN
463,464,EPIC 212737443 c,EPIC 212737443,1,1,2,Transit,2019,K2,Published Confirmed,0,<a refstr=HERATH_ET_AL__2019 href=https://ui.a...,65.55000,0.00890,-0.00890,0.0,0.28000,0.00600,-0.00600,0.0,2.69,0.146,-0.146,0.0,NaN,NaN,NaN,NaN,NaN,0.200,0.200,-0.200,0.0,NaN,NaN,NaN,NaN,316.0,10.0,-10.0,0.0,0,<a refstr=HERATH_ET_AL__2019 href=https://ui.a...,K3,4684.0,79.0,-79.0,0.0,0.67,0.02,-0.02,0.0,0.69,0.04,-0.04,0.0,-0.137,0.127,-0.127,0.0,[Fe/H],4.62,0.02,-0.02,0.0,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,13h36m53.16s,204.221507,-07d19m05.01s,-7.318058,336.25000,4.74300,-4.61500,14.810,0.137,-0.137,12.160,0.024,-0.024,14.47760,0.000386,-0.000386,2019-07-19,2019-09,2019-07-25,1.037567e+20
520,521,GJ 1132 c,GJ 1132,1,1,2,Radial Velocity,2018,La Silla Observatory,Published Confirmed,0,<a refstr=BONFILS_ET_AL__2018 href=https://ui....,8.92900,0.01000,-0.01000,0.0,0.04760,0.00170,-0.00170,0.0,NaN,NaN,NaN,NaN,2.64000,0.44000,-0.44000,0.0,Msini,0.270,NaN,NaN,1.0,NaN,NaN,NaN,NaN,300.0,5.0,-5.0,0.0,0,<a refstr=BONFILS_ET_AL__2018 href=https://ui....,NaN,3270.0,140.0,-140.0,0.0,0.21,0.01,-0.01,0.0,0.18,0.02,-0.02,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,10h14m50.18s,153.709070,-47d09m17.77s,-47.154936,12.61300,0.00700,-0.00700,13.680,0.200,-0.200,8.322,0.027,-0.027,12.13800,0.000387,-0.000387,2018-06-26,2018-10,2018-06-21,3.891993e+18
581,582,GJ 251 b,GJ 251,1,1,1,Radial Velocity,2020,W. M. Keck Observatory,Published Confirmed,0,<a refstr=STOCK_ET_AL__2020 href=https://ui.ad...,14.23800,0.00200,-0.00200,0.0,0.08180,0.00110,-0.00120,0.0,NaN,NaN,NaN,NaN,4.00000,0.40000,-0.40000,0.0,Msini,0.100,0.090,-0.070,0.0,2.53,0.04,-0.04,0.0,351.0,1.0,-1.0,0.0,0,<a refstr=STOCK_ET_AL__2020 href=https://ui.ad...,M3.0 V,3451.0,51.0,-51.0,0.0,0.36,0.01,-0.01,0.0,0.36,0.01,-0.01,0.0,-0.030,0.160,-0.160,0.0,[Fe/H],4.96,0.07,-0.07,0.0,<a refstr=STASSUN_ET_AL__2019 href=https://ui....,06h54m48.06s,103.700250,+33d15m59.27s,33.266463,5.58057,0.00198,-0.00198,9.890,0.100,-0.100,5.275,0.023,-0.023,8.85518,0.001134,-0.001134,2021-02-26,2020-11,2021-03-04,1.721996e+18
585,586,GJ 3021 b,GJ 3021,1,2,1,Radial Velocity,2000,La Silla Observatory,Published Confirmed,0,<a refstr=NAEF_ET_AL__2001 href=https://ui.ads...,133.71000,0.20000,-0.20000,0.0,0.49000,NaN,NaN,0.0,NaN,NaN,NaN,NaN,1071.04000,28.60000,-28.60000,0.0